<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5dc4ae2e6d2dd1e8f02edaea4afa234b55834c7e341e6b4ba6122afe6b534781
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 11:00:43
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 51.95 K (0.36%)
Current PnL: -16.35 L (-11.0%)
CY Booked + Current PnL: -5.07 L (-3.41%)
-------------------
Total profit:  2.29 L
Total loss:  -18.64 L
-------------------
Total Booked + Current PnL: 21.91 L (18.01%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.88%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 81.97 L (57.29%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,55.0,H-LC,10.84,172260.0,16083.0,10008.0,-0.74,10.30,5.81,16.71,22.0,1.61,1.24,29.53,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,104.48,57.0,M-SC,3.97,88111.0,-12666.0,12758.0,-0.61,-12.57,14.48,0.09,245.0,-0.99,0.64,15.76,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-43.66,65.0,H-MC,6.83,113049.0,12545.0,18099.0,0.02,12.48,16.01,30.49,79.0,0.69,0.82,48.93,MH,ATH,METALS
57,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,221036.0,10670.0,18103.0,0.22,5.07,8.19,13.68,37.0,0.59,1.59,22.43,XY25,NTT,REFINERIES
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201517.0,1379.0,19507.0,0.68,0.69,9.68,10.44,4.0,0.07,1.45,6.29,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.46,56.0,H-SC,17.06,201474.0,1520.0,95700.0,0.02,0.76,47.50,48.63,115.0,0.02,1.45,12.40,XR,NTT,MISC
30,HAVELLS,2069.16,0.20,59.0,H-MC,1.87,248508.0,777.0,74279.0,-0.03,0.31,29.89,30.30,92.0,0.01,1.79,13.65,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200700.0,-120.0,68900.0,-0.03,-0.06,34.33,34.25,80.0,-0.00,1.45,5.13,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298807.0,-17035.0,129712.0,-0.57,-5.39,43.41,35.67,5.0,-0.13,2.16,7.71,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.93,47.0,H-LC,7.84,219727.0,-32041.0,85254.0,0.13,-12.73,38.80,21.13,27.0,-0.38,1.58,18.19,X40,ATH,PAINTS
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201517.0,1379.0,19507.0,0.68,0.69,9.68,10.44,4.0,0.07,1.45,6.29,X40,NTT,FMCG
44,JIOFIN,387.00,0.47,53.0,H-LC,13.23,273050.0,5685.0,59771.0,0.52,2.13,21.89,24.48,48.0,0.10,1.97,58.27,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.79,228777.0,-5367.0,53991.0,-0.52,-2.29,23.60,20.77,86.0,-0.10,1.65,4.47,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,0.87,71.0,H-LC,1.29,249818.0,3667.0,75545.0,2.37,1.49,30.24,32.18,16.0,0.05,1.80,38.07,X200,ATH,IT
42,ITC,452.0,-37.50,52.0,H-LC,2.52,201517.0,1379.0,19507.0,0.68,0.69,9.68,10.44,4.0,0.07,1.45,6.29,X40,NTT,FMCG
51,NESTLEIND,1377.0,-7.84,59.0,H-LC,3.88,283725.0,18299.0,38501.0,0.68,6.89,13.57,21.40,11.0,0.48,2.05,14.00,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,221036.0,10670.0,18103.0,0.22,5.07,8.19,13.68,37.0,0.59,1.59,22.43,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.44,54.0,H-LC,5.78,214871.0,10371.0,31049.0,0.56,5.07,14.45,20.25,10.0,0.33,1.55,14.25,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LAOPALA,464.00,118.16,55.0,H-SC,2.34,99787.0,-31318.0,82564.0,2.54,-23.89,82.74,39.09,142.0,-0.38,0.72,28.52,AR,NTT,CERAMICS
8,ATULAUTO,844.00,3959.46,65.0,M-SC,5.79,126543.0,-20825.0,72636.0,2.40,-14.13,57.40,35.16,236.0,-0.29,0.91,29.25,XY24,NTT,AUTO
11,BANDHANBNK,400.00,-9.90,54.0,H-SC,5.13,225388.0,-53171.0,310607.0,2.39,-19.09,137.81,92.42,151.0,-0.17,1.63,30.26,XY24,NTT,BANKS
48,LTIM,7230.20,0.87,71.0,H-LC,1.29,249818.0,3667.0,75545.0,2.37,1.49,30.24,32.18,16.0,0.05,1.80,38.07,X200,ATH,IT
16,BSOFT,836.99,-11.70,59.0,H-SC,15.00,99348.0,-41305.0,114081.0,2.26,-29.37,114.83,51.74,131.0,-0.36,0.72,13.29,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89083.0,-11880.0,106410.0,-1.06,-11.77,119.45,93.63,148.0,-0.11,0.64,32.69,SR,ATH,CHEMICALS
32,HINDZINC,730.22,21.48,60.0,M-LC,1.36,197925.0,-7151.0,119725.0,-0.97,-3.49,60.49,54.89,52.0,-0.06,1.43,19.77,X5K,ATH,METALS
70,SYMPHONY,1306.00,-41.90,40.0,M-SC,18.29,123854.0,-47287.0,47238.0,-0.82,-27.63,38.14,-0.03,196.0,-1.00,0.89,1.53,OX40N,NTT,DURABLES
37,INDIAMART,4850.92,-52.03,42.0,H-SC,11.66,134154.0,10818.0,122952.0,-0.77,8.77,91.65,108.45,119.0,0.09,0.97,33.08,AR,ATH,MISC
69,SURYODAY,240.00,46.93,60.0,H-SC,2.48,134558.0,-44513.0,110243.0,-0.76,-24.86,81.93,36.71,135.0,-0.40,0.97,33.51,XR,NTT,BANKS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,104.48,57.0,M-SC,3.97,88111.0,-12666.0,12758.0,-0.61,-12.57,14.48,0.09,245.0,-0.99,0.64,15.76,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.54,124200.0,-18279.0,37794.0,0.55,-12.83,30.43,13.70,163.0,-0.48,0.90,46.59,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.06,64.0,H-SC,1.40,226980.0,-42687.0,79012.0,1.09,-15.83,34.81,13.47,138.0,-0.54,1.64,14.80,XY24,NTT,PAINTS
19,CERA,9475.0,-19.57,49.0,H-SC,2.08,146717.0,-29186.0,71202.0,0.08,-16.59,48.53,23.89,149.0,-0.41,1.06,26.71,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-18.84,57.0,H-MC,7.62,109725.0,-22110.0,64332.0,1.62,-16.77,58.63,32.03,98.0,-0.34,0.79,22.90,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-25.15,60.0,H-LC,12.33,298516.0,-47440.0,122302.0,0.09,-13.71,40.97,21.64,1.0,-0.39,2.15,5.75,X40,ATH,IT
40,INFY,2275.00,-15.29,59.0,H-LC,8.40,329575.0,16687.0,154999.0,1.64,5.33,47.03,54.87,3.0,0.11,2.38,12.34,X40,BTT,IT
42,ITC,452.00,-37.50,52.0,H-LC,2.52,201517.0,1379.0,19507.0,0.68,0.69,9.68,10.44,4.0,0.07,1.45,6.29,X40,NTT,FMCG
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298807.0,-17035.0,129712.0,-0.57,-5.39,43.41,35.67,5.0,-0.13,2.16,7.71,X40N,ATH,FMCG
1,ABB,7934.00,-37.63,68.0,H-LC,10.34,260688.0,-931.0,120151.0,0.84,-0.36,46.09,45.57,7.0,-0.01,1.88,10.93,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,63.0,M-MC,6.95,227476.0,2514.0,165148.0,-0.49,1.12,72.60,74.53,192.0,0.02,1.64,33.73,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,14.07,66.0,H-MC,3.58,197010.0,32340.0,35619.0,1.54,19.64,18.08,41.28,89.0,0.91,1.42,29.19,X40N,ATH,AC
84,VOLTAS,1918.49,1.02,58.0,H-MC,2.48,211800.0,20058.0,75973.0,0.01,10.46,35.87,50.08,99.0,0.26,1.53,17.97,XY25,ATH,AC
30,HAVELLS,2069.16,0.20,59.0,H-MC,1.87,248508.0,777.0,74279.0,-0.03,0.31,29.89,30.30,92.0,0.01,1.79,13.65,X40,ATH,ELECTRICAL
18,CAMS,5211.76,0.16,66.0,H-SC,3.83,113758.0,11754.0,32171.0,1.22,11.52,28.28,43.06,122.0,0.37,0.82,30.61,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,65.0,H-MC,6.83,113049.0,12545.0,18099.0,0.02,12.48,16.01,30.49,79.0,0.69,0.82,48.93,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,0.49,100598.0,9100.0,69654.0,-0.02,9.94,69.24,86.07,223.0,0.13,0.73,46.09,XR,NTT,DURABLES
37,INDIAMART,4850.92,-52.03,42.0,H-SC,11.66,134154.0,10818.0,122952.0,-0.77,8.77,91.65,108.45,119.0,0.09,0.97,33.08,AR,ATH,MISC
86,WIPRO,420.00,-9.26,61.0,M-LC,5.62,159669.0,8724.0,100735.0,1.33,5.78,63.09,72.51,53.0,0.09,1.15,12.15,XR,NTT,IT
12,BANKINDIA,190.00,-27.95,66.0,H-MC,6.15,184422.0,4614.0,107795.0,0.23,2.57,58.45,62.52,88.0,0.04,1.33,37.53,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.06,54325.0,-82852.0,82943.0,0.08,-60.40,152.68,0.07,267.0,-1.00,0.39,35.11,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.31,84883.0,-28666.0,68713.0,-0.27,-25.25,80.95,35.27,268.0,-0.42,0.61,107.75,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89083.0,-11880.0,106410.0,-1.06,-11.77,119.45,93.63,148.0,-0.11,0.64,32.69,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,55.0,H-SC,13.00,95130.0,-2850.0,24448.0,0.24,-2.91,25.70,22.05,152.0,-0.12,0.69,38.28,XR,ATH,FINANCE
50,NATIONALUM,247.44,-43.66,65.0,H-MC,6.83,113049.0,12545.0,18099.0,0.02,12.48,16.01,30.49,79.0,0.69,0.82,48.93,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3959.46,65.0,M-SC,5.79,126543.0,-20825.0,72636.0,2.40,-14.13,57.40,35.16,236.0,-0.29,0.91,29.25,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,67.49,72.0,H-SC,7.21,145016.0,-37759.0,148830.0,1.56,-20.66,102.63,60.77,125.0,-0.25,1.05,29.88,XR,NTT,JEWELLERY
22,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.31,84883.0,-28666.0,68713.0,-0.27,-25.25,80.95,35.27,268.0,-0.42,0.61,107.75,XR,NTT,HOTELS
61,SAMMAANCAP,326.0,-171.59,63.0,M-SC,2.63,82284.0,-19926.0,113313.0,1.53,-19.50,137.71,91.37,208.0,-0.18,0.59,33.57,XY25,NTT,FINANCE
75,TITAGARH,1548.0,1.18,69.0,H-SC,3.26,165935.0,-26080.0,104970.0,0.64,-13.58,63.26,41.08,158.0,-0.25,1.20,40.61,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.86
1,25,43.33
2,50,73.80


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    33.01
MC    22.97
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40      15.23
X40N     13.03
XR       11.63
AR        8.67
XY25      8.15
OX40N     8.15
X200      1.80
MH        1.70
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.02
H-LC    26.35
H-MC    20.05
M-SC    13.48
M-LC     5.60
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.53,-12.06,73.16
FMCG,11.74,-4.08,40.49
FINANCE,9.65,-14.97,60.00
MISC,7.42,-11.86,74.07
BANKS,6.26,-15.10,75.65
PAINTS,5.80,-14.82,31.93
ELECTRICAL,5.69,-4.35,61.86
AC,3.66,4.56,27.76
INSURANCE,3.39,-6.95,43.63


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2832442.0,22
XR,1298876.0,14
AR,1295383.0,9
X40,735468.0,10
X40N,605689.0,10
OX40N,517564.0,10
XY25,392510.0,6
SR,243771.0,2
X5K,119725.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3391847.0,29
M-SC,1376258.0,17
H-LC,1220885.0,15
H-MC,1173748.0,15
M-LC,374800.0,4
M-MC,314558.0,2
L-SC,247803.0,3
L-MC,60329.0,1
L-LC,37096.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020390.0      6
M-SC       XY24       794807.0      7
H-SC       AR         783425.0      5
           XR         751192.0      7
H-MC       XY24       488576.0      4
H-LC       X40        452907.0      5
           AR         351298.0      2
M-MC       XY24       314558.0      2
H-SC       X40N       284551.0      4
           OX40N      246266.0      4
           SR         243771.0      2
H-LC       X40N       224760.0      4
H-MC       X40        213071.0      4
           XY25       185497.0      2
L-SC       XR         164860.0      2
M-SC       AR         160660.0      2
M-LC       XY24       154340.0      2
M-SC       OX40N      124023.0      4
M-LC       X5K        119725.0      1
M-SC       XR         113965.0      2
           XY25       113313.0      1
H-MC       XR         107795.0      1
M-LC       XR         100735.0      1
H-MC       X40N        96378.0      2
L-SC       OX40N       82943.0      1
H-LC       X200        75545.0      1
M-SC       X40         69490.0      1
H-MC       OX40N       64332.0      1
H-SC       MH          62252.0      1
L-MC       XR          60329.0      1
H-LC       XY24        59771.0      1
           XY25        56604.0      2
L-LC       XY25        37096.0      1
H-MC       MH          18099.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
